Imports

In [1]:
import numpy as np
import cv2
import os
from tqdm import tqdm
from skimage.feature import hog
from sklearn.decomposition import PCA
from skimage.feature import local_binary_pattern

DATASET_PATH = "C:/temporary shit/dataset/train_data"
IMG_SIZE = (64, 64)

# RGBW

Image Processing

In [2]:
CACHE_FILE = "features_rgbw.npy"

def extract_rgbw_features(img):
    r, g, b = cv2.split(img)
    w = (r + g + b) // 3  # Approximate white channel
    return np.array([r.mean(), g.mean(), b.mean(), w.mean()])

def process_images():
    images = []
    filenames = os.listdir(DATASET_PATH)
    
    for file in tqdm(filenames, desc="Extracting RGBW"):
        img_path = os.path.join(DATASET_PATH, file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, IMG_SIZE)
        features = extract_rgbw_features(img)
        images.append(features)
    
    return np.array(images)

rgbw_features = process_images()

Extracting RGBW: 100%|██████████| 79950/79950 [03:02<00:00, 437.45it/s]


Saving Cache

In [3]:
np.save(CACHE_FILE, rgbw_features)
print(f"Saved RGBW features to {CACHE_FILE}")

Saved RGBW features to features_rgbw.npy


# HOG

Image Processing

In [4]:
CACHE_FILE = "features_hog.npy"

def extract_hog_features(img):
    features, _ = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    hist, _ = np.histogram(features, bins=32, range=(0, np.max(features)), density=True)
    return hist

hog_features = process_images()

Extracting RGBW: 100%|██████████| 79950/79950 [02:08<00:00, 624.30it/s]


Saving Cache

In [5]:
np.save(CACHE_FILE, hog_features)
print(f"Saved HOG features to {CACHE_FILE}")

Saved HOG features to features_hog.npy


# LBP

Image Processing

In [6]:
CACHE_FILE = "features_lbp.npy"

def extract_lbp_features(img):
    lbp = local_binary_pattern(img, P=8, R=1, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=59, range=(0, 59), density=True)
    return hist

lbp_features = process_images()

Extracting RGBW: 100%|██████████| 79950/79950 [02:05<00:00, 639.07it/s]


Saving Cache

In [7]:
np.save(CACHE_FILE, lbp_features)
print(f"Saved LBP features to {CACHE_FILE}")

Saved LBP features to features_lbp.npy


# Fourier Transform

Image Processing

In [8]:
CACHE_FILE = "features_fda.npy"

def extract_fourier_features(img):
    fft_img = np.fft.fft2(img)
    fft_shift = np.fft.fftshift(fft_img)
    magnitude_spectrum = np.abs(fft_shift)
    low_freq = magnitude_spectrum[:H//4, :W//4].mean()
    mid_freq = magnitude_spectrum[H//4:H//2, W//4:W//2].mean()
    high_freq = magnitude_spectrum[H//2:, W//2:].mean()
    return np.array([low_freq, mid_freq, high_freq])

fourier_features = process_images()

Extracting RGBW: 100%|██████████| 79950/79950 [02:04<00:00, 640.92it/s]


Saving Cache

In [9]:
np.save(CACHE_FILE, fourier_features)
print(f"Saved Fourier Transform features to {CACHE_FILE}")

Saved Fourier Transform features to features_fda.npy


# Error Level Analysis

Image Processing

In [10]:
CACHE_FILE = "features_ela.npy"

def extract_ela_features(img):
    cv2.imwrite("temp.jpg", img, [cv2.IMWRITE_JPEG_QUALITY, 90])
    compressed = cv2.imread("temp.jpg")
    ela = cv2.absdiff(img, compressed)
    return np.array([ela.mean()])

ela_features = process_images()

Extracting RGBW: 100%|██████████| 79950/79950 [02:02<00:00, 653.72it/s]


Saving Cache

In [11]:
np.save(CACHE_FILE, ela_features)
print(f"Saved ELA features to {CACHE_FILE}")

Saved ELA features to features_ela.npy


# Local Binary Pattern Variance

Image Processing

In [12]:
CACHE_FILE = "features_lbpv.npy"

def extract_lbpv_features(img):
    lbpv = local_binary_pattern(img, P=8, R=1, method="var")
    hist, _ = np.histogram(lbpv.ravel(), bins=59, range=(0, np.max(lbpv)), density=True)
    return hist

lbpv_features = process_images()


Extracting RGBW: 100%|██████████| 79950/79950 [02:01<00:00, 658.82it/s]


Saving Cache

In [13]:
np.save(CACHE_FILE, lbpv_features)
print(f"Saved LBPV features to {CACHE_FILE}")

Saved LBPV features to features_lbpv.npy


# Color Gamut

Image Processing

In [14]:
CACHE_FILE = "features_colorgamut.npy"

def extract_color_gamut(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return hist.flatten()

color_gamut_features = process_images()

Extracting RGBW: 100%|██████████| 79950/79950 [02:02<00:00, 651.86it/s]


Saving Cache

In [15]:
np.save(CACHE_FILE, color_gamut_features)
print(f"Saved Color Gamut features to {CACHE_FILE}")

Saved Color Gamut features to features_colorgamut.npy


# Fractal Dimension

Image Processing

In [16]:
CACHE_FILE = "features_fractal.npy"

def fractal_dimension(img, threshold=128):
    img = img < threshold
    sizes = 2 ** np.arange(1, 6)
    counts = []
    
    for size in sizes:
        S = img.shape[0] // size
        count = sum(np.any(img[i*size:(i+1)*size, j*size:(j+1)*size]) for i in range(S) for j in range(S))
        counts.append(count)

    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return np.array([-coeffs[0]])

fractal_features = process_images()


Extracting RGBW: 100%|██████████| 79950/79950 [02:01<00:00, 655.47it/s]


Saving Cache

In [17]:
np.save(CACHE_FILE, fractal_features)
print(f"Saved Fractal Dimension features to {CACHE_FILE}")

Saved Fractal Dimension features to features_fractal.npy
